In [ ]:
import numpy as np
from numpy.lib.recfunctions import append_fields
from numpy.lib.recfunctions import recursive_fill_fields
import numba

import matplotlib.pyplot as plt
import matplotlib as mpl
import uproot
import awkward as ak
import pyjet
import time

from matching import matching
import np_4vect_sum as sum4vect

plt.style.use("scientific.mplstyle")

file = uproot.open("/eos/user/c/clange/PFNano/myNanoProdMc_NANO.root")
event_tree = file['Events']

In [ ]:
max_idx = 200

pts = event_tree['PF_pt'].array(entry_stop=max_idx, library='np')
etas = event_tree['PF_eta'].array(entry_stop=max_idx, library='np')
phis = event_tree['PF_phi'].array(entry_stop=max_idx, library='np')
masses = event_tree['PF_mass'].array(entry_stop=max_idx, library='np')

pv_labels = event_tree['PF_fromPV'].array(entry_stop=max_idx, library='np')


recojet_pts = event_tree['Jet_pt'].array(entry_stop=max_idx, library='np')
recojet_etas = event_tree['Jet_eta'].array(entry_stop=max_idx, library='np')
recojet_phis = event_tree['Jet_phi'].array(entry_stop=max_idx, library='np')
recojet_masses = event_tree['Jet_mass'].array(entry_stop=max_idx, library='np')

raw_pt_factor = 1-event_tree['Jet_rawFactor'].array(entry_stop=max_idx, library='np')
pf_recojetidx = event_tree['PF_jetsIdx'].array(entry_stop=max_idx, library='np')

In [ ]:
def pyjet_jet_clustering(pts, etas, phis, masses, pv_labels, R, ptmin):
    '''Cluster jets for one event with anti kt, 15 Gev pt cut off, and CHS. It is assumed that only particles with a vertex label > 0 are signal particles'''

    event_4vect = np.column_stack((pts, etas, phis, masses)).astype('float64')
    event = np.core.records.fromarrays(event_4vect.transpose(), dtype=pyjet.DTYPE_PTEPM)
    event = append_fields(event, 'id', data=np.arange(len(event))).data
    
    chs_mask = np.where(pv_labels>0, True, False)
    event = event[chs_mask]

    sequence = pyjet.cluster(event, R=R, p=-1)
    my_jets = sequence.inclusive_jets(ptmin=ptmin)
    
    return my_jets

In [ ]:
class Node():
    '''Tree where a node represents a particle/pseudojet in the clustering process. 
    Leaf nodes represent PF candidates of an event, s.t. their label is the index of the PF candidate.
    The parents if a node are the two particles/pseudojets of the previous clustering stage which have been merged to form a new particle.'''
    
    def __init__(self, label, is_PF = False):
        self.label = label
        self.is_leaf = is_PF
        self.parents = []
        
    def add_parents(self, parent_node1, parent_node2):
        self.parents = [parent_node1, parent_node2]
        
    def get_leaf_labels(self):
        '''Return PF candiates which have been clustered to form this Node'''
        leaf_labels = []
        self._collect_leaf_nodes(self, leaf_labels)
        return leaf_labels
    
    def _collect_leaf_nodes(self, node, leaf_labels):
        if node.is_leaf:
            leaf_labels.append(node.label)
        for n in node.parents:
            self._collect_leaf_nodes(n, leaf_labels)

In [ ]:
def rapidity(pt, eta, m):
    '''Accepts np.array or scalars for all inputs and returns rapidity in the format of the input'''
    return np.log((np.sqrt(m**2+pt**2*np.cosh(eta)**2) + pt*np.sinh(eta))/np.sqrt(m**2+pt**2))


@numba.vectorize
def anti_kt_dist(pt_i, pt_j, y_i, y_j, phi_i, phi_j, R):
    dphi = phi_i - phi_j
    dR2 = (y_i-y_j)**2 + ((dphi+np.pi)%(2*np.pi)-np.pi)**2
    
    return min(1/pt_i**2, 1/pt_j**2)*dR2/R**2
    

def myown_jet_clustering(pts, etas, phis, ms, pv_labels, R, ptmin):
    jet = [] # 4 vectors of clustered jets
    
    # make structured array (pt, eta, phi, m, y, node) for every particle in event
    # collection of structured particles stored in event
    ys = rapidity(pts, etas, ms)
    id_nodes = [Node(i, is_PF=True) for i in range(len(pts))]
    event_4vect = np.column_stack((pts, etas, phis, ms)).astype('float64')
    event = np.core.records.fromarrays(event_4vect.transpose(), dtype=pyjet.DTYPE_PTEPM)
    event = append_fields(event, 'y', data=ys, dtypes='f8').data
    # make new data type to add field with Node instances
    dt = event.dtype.descr + [('id_node', 'O')] 
    aux = np.empty(event.shape, dt) 
    event = recursive_fill_fields(event, aux)
    event['id_node'] = id_nodes
    
    chs_mask = np.where(pv_labels>0)[0]
    event = event[chs_mask]
    
    
    while len(event) > 1:
        pairs = np.triu_indices(len(event),k=1) # tuple of two array representing all valid combinations of particle i and j 
        anti_kt_distance = anti_kt_dist(event['pT'][pairs[0]], event['pT'][pairs[1]], event['y'][pairs[0]], event['y'][pairs[1]], event['phi'][pairs[0]], event['phi'][pairs[1]], R)
        beam_distance = 1/event['pT']**2
        
        # PROBLEM: CAN HAVE SEVERAL OCCURENCES OF MIN VALUE; WHAT DOES FASTJET DO?
        min_anti_kt_d = np.min(anti_kt_distance)
        anti_kt_dmin_pair_idxs = np.where(anti_kt_distance==min_anti_kt_d)[0]
        # choose particle pair with max pt
        if len(anti_kt_dmin_pair_idxs) > 1:
            pts_potential_pairs = event['pT'][pairs[0][anti_kt_dmin_pair_idxs]] + event['pT'][pairs[1][anti_kt_dmin_pair_idxs]]
            chosen_pair_id = np.argmax(pts_potential_pairs)
            min_id = anti_kt_dmin_pair_idxs[chosen_pair_id]
        else:
            min_id = anti_kt_dmin_pair_idxs[0]
            
        min_ij = [pairs[0][min_id], pairs[1][min_id]] # guarentees smaller index always first  
       
        # PROBLEM: CAN HAVE SEVERAL OCCURENCES OF MIN VALUE; WHAT DOES FASTJET DO?
        min_beam_d = np.min(beam_distance)
        beam_dmin_idxs = np.where(beam_distance==min_beam_d)[0]
        # choose particle with larger mass 
        if len(beam_dmin_idxs) > 1:
            chosen_min = np.argmax(event['mass'][beam_dmin_idxs])
            min_idx = beam_dmin_idxs[chosen_min]
        else:
            min_idx = beam_dmin_idxs[0]
            
        
        if (min_beam_d < min_anti_kt_d): # declare particle as jet if pt>=ptmin
            if event[min_idx]['pT'] >= ptmin:
                jet.append(event[min_idx])
            event = np.delete(event, min_idx, axis=0)
        else: # merge two particles together
            # find new 4 vector
            p1, p2 = event[min_ij[0]].tolist()[:4], event[min_ij[1]].tolist()[:4]
            aux = np.column_stack((p1,p2))
            new_vec4 = sum4vect.np_sum4vec(aux[0], aux[1], aux[2], aux[3])
            new_y = rapidity(new_vec4[0], new_vec4[1], new_vec4[3])
            # make new node
            parent_node1, parent_node2 = event['id_node'][min_ij[0]], event['id_node'][min_ij[1]] 
            new_node = Node(label=parent_node1.label) # smaller of the two parent labels
            new_node.add_parents(parent_node1, parent_node2)
            # change particle with smaller id to merged one and delete particle with larger id
            event[min_ij[0]] = tuple(np.concatenate((new_vec4, [new_y, new_node])))
            event = np.delete(event, min_ij[1], axis=0)
            
            
    # deal with last particle separtely as anti-kt-distance meaningless for single particle 
    if event[0]['pT'] >= ptmin:
        jet.append(event[0])
    event = np.delete(event, 0, axis=0)
    
    # deal with case when no jets exist
    if len(jet) == 0:
        return [], [] 
    
    # PROBLEM: EVENT 67 THROWS ERROR WHEN SORTING: '<' not supported between instances of 'Node' and 'Node'. Fixed when including except TypeError
    try:
        jet = np.sort(np.array(jet), order=['pT', 'mass'])[::-1] # sort by decending pt and if two jets have same pt, sort those my mass
    except TypeError:
        jet_pts = [jet[0] for jet in jet]
        sorting_idxs = np.argsort(jet_pts)
        jet = np.array(jet)[sorting_idxs]

    # find PF constituents of every jet
    jets = np.empty(len(jet), dtype=pyjet.DTYPE_PTEPM)
    jet_consti = []
    for i, jet in enumerate(jet):
        jet_consti.append(np.sort(jet['id_node'].get_leaf_labels()))
        jets[i] = jet[['pT', 'eta', 'phi', 'mass']]
        
    
    return jets, jet_consti

### Compare jets from my algorithm to the ones from pyjet and to recojets

In [ ]:
ptmin = 15
jet_parameter = 0.4
cluster_time_myalgo, cluster_time_pyjet = [], []
same_consti_pyjet, same_consti_recojet = [], [] # each element will be boolean list where each entry states wheter constituents of myjet are the same as in pyjet/recojet
# Difference of my clustered jets to jets clustered by pyjet and the recojets. pt and mass are normalized to pyjet/reco jet quantaties
dpt_pyjet, dR_pyjet, dmass_pyjet = [], [], []
dpt_recojet, dR_recojet, dmass_recojet = [], [], []
dn_recojet = []

for i in range(max_idx):
    #print("Event ", i) # indicating which event is currently clustered
    t1 = time.time()
    pyjetjets = pyjet_jet_clustering(pts[i], etas[i], phis[i], masses[i], pv_labels[i], jet_parameter, ptmin)
    pyjets_consti = []
    for jet in pyjetjets:
        pyjets_consti.append(np.sort([consti.id for consti in jet]))
    t2 = time.time()
    cluster_time_pyjet.append(t2-t1)

    t3 = time.time()
    myjets, myjets_consti = myown_jet_clustering(pts[i], etas[i], phis[i], masses[i], pv_labels[i], jet_parameter, ptmin)
    t4 = time.time()
    cluster_time_myalgo.append(t4-t3)
    

    if len(myjets)==0:
        continue
    
    # find differences to pyjetjets
    pyjet_etas = [pyjetjet.eta for pyjetjet in pyjetjets]
    pyjet_phis = [pyjetjet.phi for pyjetjet in pyjetjets]
    matches_pyjet = matching(myjets['eta'], myjets['phi'], pyjet_etas, pyjet_phis)
    same_consti_this_event = []
    
    for p,q in enumerate(matches_pyjet): # (p,q) is tuple of myjet index and pyjet index, s.t. jet axis have minmal separation.
        if q == -1:
            print("Couldn't find fitting pyjet to myjet {} in event {}".format(p, i))
        else:
            dpt_pyjet.append((myjets[p]['pT'] - pyjetjets[q].pt)/pyjetjets[q].pt)
            dR_pyjet.append(np.sqrt((myjets[p]['eta'] - pyjetjets[q].eta)**2 + (myjets[p]['phi'] - pyjetjets[q].phi)**2))
            dmass_pyjet.append((myjets[p]['mass'] - pyjetjets[q].mass)/pyjetjets[q].mass)
            
            if len(myjets_consti[p]) == len(pyjets_consti[q]):
                same_consti_this_event.append(np.allclose(myjets_consti[p], pyjets_consti[q]))
            else:
                same_consti_this_event.append(False)
                print("Different number of constituents found in myjet {}, pyjet {} of event {}.\n\tMy algo: {} constituents\n\tPyjet : {} constituents".format(p, q, i, len(myjets_consti[p]), len(pyjets_consti[q])))
                
    same_consti_pyjet.append(same_consti_this_event)
    
    
    # find differences to recojets
    raw_reco_pts, raw_reco_m = recojet_pts[i]*raw_pt_factor[i], recojet_masses[i]*raw_pt_factor[i]
    reco_etas, reco_phis = recojet_etas[i], recojet_phis[i]
    reco_consti = []
    for recojet_idx in np.unique(pf_recojetidx[i])[1:]:
        reco_consti.append(np.where(pf_recojetidx[i]==recojet_idx)[0])
    
    matches_recojet = matching(myjets['eta'], myjets['phi'], reco_etas, reco_phis)
    same_consti_this_event = []
    n_unpaired_myjets = 0
    
    for p,q in enumerate(matches_recojet):
        if q == -1:
            n_unpaired_myjets += 1
        else:
            dpt_recojet.append((myjets[p]['pT'] - raw_reco_pts[q])/raw_reco_pts[q])
            dR_recojet.append(np.sqrt((myjets[p]['eta'] - reco_etas[q])**2 + (myjets[p]['phi'] - reco_phis[q])**2))
            dmass_recojet.append((myjets[p]['mass'] - raw_reco_m[q])/raw_reco_m[q])
            
            if len(myjets_consti[p]) == len(reco_consti[q]):
                same_consti_this_event.append(np.allclose(myjets_consti[p], reco_consti[q]))
            else:
                same_consti_this_event.append(False)
                print("Different number of constituents found in myjet {}, recojet {} of event {}.\n\tMy algo: {} constituents\n\tRecojet : {} constituents".format(p, q, i, len(myjets_consti[p]), len(reco_consti[q])))
                
    same_consti_recojet.append(same_consti_this_event)
    dn_recojet.append((i,n_unpaired_myjets)) # (event id, number of excess myjets)

In [ ]:
py_xlabels = ['$\\frac{myjet_{pt} - pyjet_{pt}}{pyjet_{pt}}$', '$\\frac{myjet_{m} - pyjet_{m}}{pyjet_{m}}$']
reco_xlabels = ['$\\frac{myjet_{pt} - recojet_{pt}}{recojet_{pt}}$', '$\\frac{myjet_{m} - recojet_{m}}{recojet_{m}}$']
img_names = ["plots/mycluster_algo_vs_pyjet.pdf", "plots/mycluster_algo_vs_recojet.pdf"]

def plot_differences(dR, dpt, dm, same_consti, xlabels, img_name):
    fig, ax = plt.subplots(1,3, figsize=(26, 7))
    
    n_jets = len(dR)
    n_false_consti = 0
    for same_c in same_consti:
        n_false_consti += same_c.count(False)
    
    ax[0].hist(dR, bins=50, histtype='step')
    ax[0].text(0.4, 0.9, "# considered events: {:.0f}\n# compared jets: {:.0f}\nfrequency of correct constituents: {:.2%}".format(max_idx, n_jets, 1-n_false_consti/n_jets), ha='left', va='top', transform = ax[0].transAxes)
    ax[0].set_title('Separation of jet axis')
    ax[0].set_yscale('log')
    ax[0].set_xlabel('$\Delta R$')
    ax[0].set_ylabel('Counts')
    
    ax[1].hist(dpt, bins=50, histtype='step')
    ax[1].set_title('Normalized $\Delta pt$')
    ax[1].set_yscale('log')
    ax[1].set_xlabel(xlabels[0])
    ax[1].set_ylabel('Counts')
    
    ax[2].hist(dm, bins=50, histtype='step')
    ax[2].set_title('Normalized $\Delta m$')
    ax[2].set_yscale('log')
    ax[2].set_xlabel(xlabels[1])
    ax[2].set_ylabel('Counts')
    
    #fig.savefig(img_name)

In [ ]:
plot_differences(dR_pyjet, dpt_pyjet, dmass_pyjet, same_consti_pyjet, py_xlabels, img_names[0])

In [ ]:
plot_differences(dR_recojet, dpt_recojet, dmass_recojet, same_consti_recojet, reco_xlabels, img_names[1])

In [ ]:
plt.plot(np.arange(max_idx), cluster_time_myalgo, label='my algo')
plt.plot(np.arange(max_idx), cluster_time_pyjet, label='pyjet')
plt.legend()
plt.title('Comparison of cluster time')
plt.xlabel('Event id')
plt.ylabel('Cluster time in sec')
plt.show()

In [ ]:
print("My Algo: Total: {:.3f} sec\t mean per event: {:.3f} sec\nPyjet: Total: {:.3f} sec\t mean per event: {:.3f} sec".format(np.sum(cluster_time_myalgo), np.mean(cluster_time_myalgo), np.sum(cluster_time_pyjet), np.mean(cluster_time_pyjet)))